# Index Type Comparison: Flat vs IVF

This notebook compares the performance of **Flat** (brute force) and **IVF** (Inverted File) indexes.

**Flat Index:**
- Brute force search using cosine similarity
- O(n) search complexity
- 100% recall accuracy
- Best for smaller datasets (<100k vectors)

**IVF Index:**
- Clusters vectors using k-means
- Searches only relevant clusters (nprobe)
- Faster search, but approximate results
- Better for larger datasets

**Setup:** Start the server with `make start` before running this notebook.

In [1]:
import sys
sys.path.insert(0, "..")

import time
from client import StackAIClient

client = StackAIClient()
client.print_health()

Server is running


## Configuration

Choose a preset or customize parameters:

| Preset | Chunks | Est. Time (per index) | Use Case |
|--------|--------|----------------------|----------|
| `small` | 100 | ~1 sec | Quick sanity check |
| `medium` | 500 | ~4 sec | See differences emerge |
| `large` | 10,000 | ~80 sec | Clear IVF search advantage |
| `xlarge` | 100,000 | ~13 min | Stress test |

*Times based on ~128 chunks/sec with Cohere embedding API.*

In [ ]:
# === CHOOSE A PRESET ===
PRESET = "small"  # Options: "small", "medium", "large", "xlarge", "custom"

# Preset configurations
PRESETS = {
    "small":  {"chunks": 100,   "batch": 50,  "queries": 10},
    "medium": {"chunks": 500,   "batch": 100, "queries": 20},
    "large":  {"chunks": 10000,  "batch": 200, "queries": 30},
    "xlarge": {"chunks": 100000,  "batch": 500, "queries": 50},
    "custom": {"chunks": 100,   "batch": 50,  "queries": 10},  # Edit these
}

# Apply preset (or use custom values)
config = PRESETS[PRESET]
NUM_CHUNKS = config["chunks"]
BATCH_SIZE = config["batch"]
NUM_QUERIES = config["queries"]
K_VALUES = [5, 10, 20]

print(f"Preset: {PRESET.upper()}")
print(f"  Chunks per index: {NUM_CHUNKS:,}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Search queries: {NUM_QUERIES}")
print(f"  K values: {K_VALUES}")

# Estimate time based on ~128 chunks/sec (Cohere batching at 96 texts/call)
CHUNKS_PER_SEC = 128
est_seconds = (NUM_CHUNKS * 2) / CHUNKS_PER_SEC  # x2 for both indexes
if est_seconds < 60:
    print(f"\nEstimated creation time: ~{est_seconds:.0f} seconds")
else:
    print(f"\nEstimated creation time: ~{est_seconds / 60:.1f} minutes")

## Sample Data

Generate sample text chunks for testing:

In [ ]:
# Sample texts about various topics
TOPICS = [
    "Machine learning algorithms process data to find patterns and make predictions.",
    "Neural networks are inspired by biological brain structures and connections.",
    "Deep learning uses multiple layers to extract hierarchical features from data.",
    "Natural language processing enables computers to understand human language.",
    "Computer vision systems can identify objects and scenes in images.",
    "Reinforcement learning trains agents through rewards and penalties.",
    "Transformers use attention mechanisms for sequence processing tasks.",
    "Vector databases store embeddings for fast similarity search.",
    "Clustering algorithms group similar data points together.",
    "Dimensionality reduction compresses high-dimensional data.",
    "Gradient descent optimizes model parameters iteratively.",
    "Convolutional networks excel at processing grid-like data.",
    "Recurrent networks handle sequential data with memory.",
    "Attention mechanisms help models focus on relevant information.",
    "Transfer learning reuses knowledge from pre-trained models.",
    "Data augmentation increases training set diversity.",
    "Regularization techniques prevent model overfitting.",
    "Batch normalization stabilizes neural network training.",
    "Dropout randomly deactivates neurons during training.",
    "Cross-validation estimates model generalization performance.",
]

# Generate chunks by cycling through topics with variations
def generate_chunks(n: int) -> list[dict]:
    chunks = []
    for i in range(n):
        topic = TOPICS[i % len(TOPICS)]
        text = f"[Chunk {i+1}] {topic} This is variation {i // len(TOPICS) + 1}."
        chunks.append({"text": text, "metadata": {"index": i}})
    return chunks

# Sample queries
QUERIES = [
    "How does machine learning work?",
    "What are neural networks?",
    "Explain deep learning",
    "How do transformers process text?",
    "What is computer vision?",
    "How does clustering work?",
    "What is gradient descent?",
    "Explain attention mechanisms",
    "What is transfer learning?",
    "How to prevent overfitting?",
]

print(f"Generated {len(TOPICS)} topic templates")
print(f"Sample queries: {len(QUERIES)}")

## Create Test Libraries

Create one library with Flat index and one with IVF index:

In [ ]:
# Create libraries with different index types
flat_lib = client.create_library("Index Comparison - Flat", index_type="flat")
ivf_lib = client.create_library("Index Comparison - IVF", index_type="ivf")

print(f"Created Flat library: ID={flat_lib['id']}, index_type={flat_lib['index_type']}")
print(f"Created IVF library: ID={ivf_lib['id']}, index_type={ivf_lib['index_type']}")

# Create documents
flat_doc = client.create_document(flat_lib['id'], "Test Document")
ivf_doc = client.create_document(ivf_lib['id'], "Test Document")

print(f"Created documents: flat_doc={flat_doc['id']}, ivf_doc={ivf_doc['id']}")

## Benchmark: Chunk Creation

Compare the time to create and index chunks:

In [ ]:
chunks = generate_chunks(NUM_CHUNKS)
print(f"Generated {len(chunks)} chunks")

# Time Flat index chunk creation
print("\n--- Flat Index: Creating chunks ---")
flat_start = time.perf_counter()
for i in range(0, len(chunks), BATCH_SIZE):
    batch = chunks[i:i + BATCH_SIZE]
    client.create_chunks_batch(flat_doc['id'], batch)
    print(f"  Batch {i // BATCH_SIZE + 1}: {len(batch)} chunks")
flat_create_time = time.perf_counter() - flat_start
print(f"Flat index creation time: {flat_create_time:.2f}s")

# Time IVF index chunk creation
print("\n--- IVF Index: Creating chunks ---")
ivf_start = time.perf_counter()
for i in range(0, len(chunks), BATCH_SIZE):
    batch = chunks[i:i + BATCH_SIZE]
    client.create_chunks_batch(ivf_doc['id'], batch)
    print(f"  Batch {i // BATCH_SIZE + 1}: {len(batch)} chunks")
ivf_create_time = time.perf_counter() - ivf_start
print(f"IVF index creation time: {ivf_create_time:.2f}s")

print(f"\n=== Creation Summary ===")
print(f"Flat: {flat_create_time:.2f}s ({NUM_CHUNKS / flat_create_time:.1f} chunks/sec)")
print(f"IVF:  {ivf_create_time:.2f}s ({NUM_CHUNKS / ivf_create_time:.1f} chunks/sec)")

## Benchmark: Search Performance

Compare search times across different k values:

In [ ]:
import statistics

def benchmark_search(library_id: int, queries: list[str], k: int, num_runs: int = 1) -> list[float]:
    """Run searches and return timing results."""
    times = []
    for query in queries[:num_runs]:
        start = time.perf_counter()
        client.search(library_id, query, k=k)
        elapsed = (time.perf_counter() - start) * 1000  # ms
        times.append(elapsed)
    return times

results = {"flat": {}, "ivf": {}}

for k in K_VALUES:
    print(f"\n--- Benchmarking k={k} ---")
    
    flat_times = benchmark_search(flat_lib['id'], QUERIES, k, NUM_QUERIES)
    ivf_times = benchmark_search(ivf_lib['id'], QUERIES, k, NUM_QUERIES)
    
    results["flat"][k] = flat_times
    results["ivf"][k] = ivf_times
    
    print(f"  Flat: avg={statistics.mean(flat_times):.1f}ms, min={min(flat_times):.1f}ms, max={max(flat_times):.1f}ms")
    print(f"  IVF:  avg={statistics.mean(ivf_times):.1f}ms, min={min(ivf_times):.1f}ms, max={max(ivf_times):.1f}ms")

## Results Summary

In [ ]:
print("=" * 60)
print("INDEX COMPARISON RESULTS")
print("=" * 60)
print(f"\nPreset: {PRESET.upper()}")
print(f"Dataset: {NUM_CHUNKS:,} chunks per index")
print(f"Queries: {NUM_QUERIES} per k value")

print("\n--- Creation Time ---")
print(f"  Flat: {flat_create_time:.2f}s ({NUM_CHUNKS / flat_create_time:.1f} chunks/sec)")
print(f"  IVF:  {ivf_create_time:.2f}s ({NUM_CHUNKS / ivf_create_time:.1f} chunks/sec)")
diff = abs(flat_create_time - ivf_create_time)
faster = "Flat" if flat_create_time < ivf_create_time else "IVF"
print(f"  Winner: {faster} by {diff:.2f}s")

print("\n--- Search Time (avg ms) ---")
print(f"{'k':>5} | {'Flat':>10} | {'IVF':>10} | {'Speedup':>10}")
print("-" * 45)
for k in K_VALUES:
    flat_avg = statistics.mean(results['flat'][k])
    ivf_avg = statistics.mean(results['ivf'][k])
    if ivf_avg > 0 and flat_avg > 0:
        if ivf_avg < flat_avg:
            speedup = f"IVF {flat_avg/ivf_avg:.1f}x"
        else:
            speedup = f"Flat {ivf_avg/flat_avg:.1f}x"
    else:
        speedup = "-"
    print(f"{k:>5} | {flat_avg:>10.1f} | {ivf_avg:>10.1f} | {speedup:>10}")

print("\n--- Interpretation ---")
if NUM_CHUNKS < 500:
    print("- Small dataset: Flat may be faster (less overhead)")
    print("- Try 'medium' or 'large' preset to see IVF benefits")
elif NUM_CHUNKS < 2000:
    print("- Medium dataset: IVF should start showing advantages")
    print("- Search speedup becomes more noticeable")
else:
    print("- Large dataset: IVF search should be significantly faster")
    print("- Trade-off: IVF has slightly lower recall accuracy")

## Search Result Comparison

Compare actual search results to check recall accuracy:

In [ ]:
# Compare search results for a specific query
test_query = "How does machine learning work?"
k = 5

print(f'Query: "{test_query}"')
print(f"k={k}")

flat_results = client.search(flat_lib['id'], test_query, k=k)
ivf_results = client.search(ivf_lib['id'], test_query, k=k)

print("\n--- Flat Index Results ---")
for i, r in enumerate(flat_results['results'], 1):
    print(f"  {i}. [score={r['score']:.4f}] {r['chunk']['text'][:60]}...")

print("\n--- IVF Index Results ---")
for i, r in enumerate(ivf_results['results'], 1):
    print(f"  {i}. [score={r['score']:.4f}] {r['chunk']['text'][:60]}...")

# Calculate overlap by TEXT content (not ID, since they're in different libraries)
flat_texts = set(r['chunk']['text'] for r in flat_results['results'])
ivf_texts = set(r['chunk']['text'] for r in ivf_results['results'])
overlap = len(flat_texts & ivf_texts)
print(f"\nResult overlap: {overlap}/{k} chunks have identical text")
print(f"IVF recall vs Flat: {overlap/k*100:.0f}%")

if overlap < k:
    print(f"\nNote: {k - overlap} results differ. This is expected with IVF -")
    print("it searches fewer clusters for speed, trading some accuracy.")

## Cleanup

Delete the test libraries:

In [ ]:
client.delete_library(flat_lib['id'])
client.delete_library(ivf_lib['id'])
print("Test libraries deleted")